<a href="https://colab.research.google.com/github/tessaSlice/mhacks-travel-app/blob/main/MHacks_get_images_of_city.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Downloading & accessing the GEMINI API

Note: if you are encountering an issue where it does not allow you to access the API, then check the location you are currently at with this command. If it is in the list of countries that are not supported, then reset the runtime and reconnect.

In [ ]:
!curl ipinfo.io

{
  "ip": "35.232.69.39",
  "hostname": "39.69.232.35.bc.googleusercontent.com",
  "city": "Council Bluffs",
  "region": "Iowa",
  "country": "US",
  "loc": "41.2619,-95.8608",
  "org": "AS396982 Google LLC",
  "postal": "51502",
  "timezone": "America/Chicago",
  "readme": "https://ipinfo.io/missingauth"
}

In [ ]:
import os
from google.colab import userdata
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

### Using system instructions & set system response type to JSON

In [ ]:
%%bash

curl https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro-latest:generateContent?key=$GOOGLE_API_KEY \
    -H 'Content-Type: application/json' \
    -X POST \
    -d '{ "contents":[{
            "parts":[{"text": "List 10 popular tourist destinations using this JSON schema: \{ \"City name\": \"object\", \"properties\": \{ \"Entertainment\": \{ \"type\": \"string\" \},\}\}"}] }],
          "generationConfig": {
            "response_mime_type": "application/json",
          } }'

{
  "candidates": [
    {
      "content": {
        "parts": [
          {
            "text": "[\n  {\n    \"City name\": {\n      \"properties\": {\n        \"Entertainment\": {\n          \"type\": \"string\"\n        }\n      }\n    }\n  },\n  {\n    \"Paris\": {\n      \"properties\": {\n        \"Entertainment\": {\n          \"type\": \"string\"\n        }\n      }\n    }\n  },\n  {\n    \"London\": {\n      \"properties\": {\n        \"Entertainment\": {\n          \"type\": \"string\"\n        }\n      }\n    }\n  },\n  {\n    \"Rome\": {\n      \"properties\": {\n        \"Entertainment\": {\n          \"type\": \"string\"\n        }\n      }\n    }\n  },\n  {\n    \"New York City\": {\n      \"properties\": {\n        \"Entertainment\": {\n          \"type\": \"string\"\n        }\n      }\n    }\n  },\n  {\n    \"Barcelona\": {\n      \"properties\": {\n        \"Entertainment\": {\n          \"type\": \"string\"\n        }\n      }\n    }\n  },\n  {\n    \"Dubai\": {\n   

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2469    0  2155  100   314    202     29  0:00:10  0:00:10 --:--:--   633


## Use python to interact with the REST API

In [ ]:
!pip install requests

In [ ]:
import requests

google_API_key = userdata.get('GOOGLE_API_KEY')

url = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro-latest:generateContent?key={google_API_key}"
headers = {'Content-Type' : 'application/json'}

city_names_prompt = "list 10 popular tourist destinations using this JSON schema: \
            \{ \"City name\": \
            \"object\", \
            \}\}"

simplified_city_names_prompt = "You are an amazing and well known traveling advisor. List 10 popular tourist destinations."

data = {
    'contents': [{
        'parts': [{
            "text": simplified_city_names_prompt,
        }]
    }],
    "generationConfig": {
        "response_mime_type": "application/json",
    }
}

response = requests.post(url, headers=headers, json=data)

print(response.status_code)
print(response.json())

200
{'candidates': [{'content': {'parts': [{'text': '[\n  "Paris, France",\n  "Rome, Italy",\n  "London, England",\n  "New York City, USA",\n  "Barcelona, Spain",\n  "Dubai, United Arab Emirates",\n  "Bangkok, Thailand",\n  "Singapore",\n  "Tokyo, Japan",\n  "Sydney, Australia"\n]\n\n'}], 'role': 'model'}, 'finishReason': 'STOP', 'index': 0, 'safetyRatings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE'}]}]}


### Write the entire JSON response into a text file

In [ ]:
import json

file_path = "json.txt"

with open(file_path, 'w') as file:
  # write the JSON into the file
  json.dump(response.json(), file)


### Get only the relevant part of the JSON, which is the text attribute

In [ ]:
text = response.json()['candidates'][0]['content']['parts'][0]['text']
print(text)

[
  "Paris, France",
  "Rome, Italy",
  "London, England",
  "New York City, USA",
  "Barcelona, Spain",
  "Dubai, United Arab Emirates",
  "Bangkok, Thailand",
  "Singapore",
  "Tokyo, Japan",
  "Sydney, Australia"
]




In [ ]:
# write it into the json
import json

file_path = "response.json"

with open(file_path, 'w') as file:
  # write the JSON into the file
  file.write(text)

# Using the simple list of city names, grab links to photos from Google

Using requests again to do it now

In [ ]:
# Query for image URL
def get_image_url(query, num_images):
  # Get image URLs
  image_urls = get_google_images(query, api_key, cse_id, num_images)

  # Print image URLs
  if image_urls:
      for url in image_urls:
          return url
  else:
      return "No images found."

In [ ]:
import requests

def get_google_images(query, api_key, cse_id, num_images=10):
    url = f"https://www.googleapis.com/customsearch/v1?"
    params = {
        "q": query,
        "searchType": "image",
        "key": api_key,
        "cx": cse_id,
        "num": num_images
    }

    response = requests.get(url, params=params)
    data = response.json()

    if 'items' in data:
        image_urls = [item['link'] for item in data['items']]
        return image_urls
    else:
        return None

In [ ]:
import json

# Set up your Google API credentials
# apparently I need a custom API key seperate from GEMINI to do it as well
api_key = userdata.get('CUSTOM_ENGINE_SEARCH_KEY')
cse_id = userdata.get('SEARCH_ENGINE_ID')

# loop through the different city locations
city_image_urls = []

dictionary = json.loads(text)

for city_name in dictionary:
  city_image_urls.append(get_image_url(city_name, 1))

for url in city_image_urls:
  print(url)

https://www.ciee.org/sites/default/files/styles/640x590/public/images/2023-05/paris-aerial-arc-de-triomphe.jpg?h=4640d74d&itok=N_VbzQG8
https://i.natgeofe.com/n/3012ffcc-7361-45f6-98b3-a36d4153f660/colosseum-daylight-rome-italy_3x4.jpg
https://cdn.britannica.com/94/19694-050-1DAF09F3/World-Heritage-locator-London-United-Kingdom.jpg
https://www.cunard.com/content/dam/cunard/inventory-assets/ports/NYC/NYC.jpg.1706023244423.image.750.563.low.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/5/56/Aerial_view_of_Barcelona%2C_Spain_%2851227309370%29_%28cropped%29.jpg/640px-Aerial_view_of_Barcelona%2C_Spain_%2851227309370%29_%28cropped%29.jpg
https://a.cdn-hotels.com/gdcs/production121/d772/6b5a9a4c-fd06-4bcf-b2f0-d979e3704cf9.jpg
https://a.cdn-hotels.com/gdcs/production172/d459/3af9262b-3d8b-40c6-b61d-e37ae1aa90aa.jpg?impolicy=fcrop&w=800&h=533&q=medium
https://www.state.gov/wp-content/uploads/2023/07/shutterstock_1932196766v2.jpg
https://media.cntraveler.com/photos/63482b255e7943ad40

In [ ]:
# data_dict is a dictionary
data_dict = {}

for city_name in dictionary:
  data_dict[city_name] = city_name

data_str = json.dumps(data_dict, ensure_ascii=False)

cities = json.loads(data_str)
url_index = 0
for city in cities:
    city_name = city["city_name"]
    # add the city image url to the dictionary
    city["image_url"] = city_image_urls[url_index]
    url_index = url_index + 1

# # cities should be a text, seems like jsonify doesn't support dicts
# cities_text = json.dumps(cities)

cities

{'city_name': 'Sydney, Australia'}

In [ ]:
# city_image_urls now holds the urls for the different cities to visit!